In [ ]:
import json
from datetime import datetime
from jsonpath_ng.ext import parse

In [ ]:
kinds_platforms = {
#    'linux':    'generic',
#    'ceos':     'eos',
#    'crpd':     'generic', 
    'srl':      'srlinux',
#    'vr-veos':  'eos',
#    'vr-vmx':   'junos', 
#    'vr-xrv9k': 'iosxr',
}

kinds_params = {
    'linux': {
      'hostname': "cat /proc/sys/kernel/hostname",
      'domainname': "cat /proc/sys/kernel/domainname",
      'vendor': "source /etc/lsb-release; echo $DISTRIB_ID",
      'model': "echo $DISTRIB_RELEASE",
      'serial_number': "cat /sys/devices/virtual/dmi/id/product_serial",
      'version': "cat /proc/version",
      'uptime': "cat /proc/uptime",
      'interfaces': "ip -json address show",
    },
    'ceos': {
        'facts': "facts",
        'interfaces': "interfaces",
        'interfaces_ip': "interfaces_ip",
        'lldp_neighbors': "lldp_neighbors",
    },
    'crpd': {
      'hostname': "cat /proc/sys/kernel/hostname",
      'domainname': "cat /proc/sys/kernel/domainname",
      'vendor': "source /etc/lsb-release; echo $DISTRIB_ID",
      'model': "echo $DISTRIB_RELEASE",
      'serial_number': "cat /sys/devices/virtual/dmi/id/product_serial",
      'version': "cat /proc/version",
      'uptime': "cat /proc/uptime",
      'interfaces': "ip -json address show",
    },
    'srl': {
        #'name':      "srl_nokia-system:system/srl_nokia-system-name:name",
        'hostname':      "srl_nokia-system:system/srl_nokia-system-name:name/host-name",
        'domainname':    "srl_nokia-system:system/srl_nokia-system-name:name/domain-name",
        'os_version':    "srl_nokia-system:system/srl_nokia-system-info:information/version",
        'model':         "srl_nokia-platform:platform/srl_nokia-platform-chassis:chassis/type",
        'serial_number': "srl_nokia-platform:platform/srl_nokia-platform-chassis:chassis/serial-number",
        'last_booted':   "srl_nokia-system:system/srl_nokia-system-info:information/last-booted",
        'interfaces':    "srl_nokia-interfaces:interface",
        #'lldp_neighbors': "srl_nokia-system:system/srl_nokia-lldp:lldp/interface",
    }
}

kinds_jsonpaths = {
    "srl": {
        "hostname": {
            "query":     "$.updates[?(path=='srl_nokia-system:system/srl_nokia-system-name:name/host-name')].val",
        },
        "domainname": {
            "query":     "$.updates[?(path=='srl_nokia-system:system/srl_nokia-system-name:name/domain-name')].val",
            
        },
        "os_version": {
            "query":     "$.updates[?(path=='srl_nokia-system:system/srl_nokia-system-info:information/version')].val",
            
        },
        "model": {
            "query":     "$.updates[?(path=='srl_nokia-platform:platform/srl_nokia-platform-chassis:chassis/type')].val",
            
        },
        "serial_number": {
            "query":     "$.updates[?(path=='srl_nokia-platform:platform/srl_nokia-platform-chassis:chassis/serial-number')].val",
            
        },
        "last_booted": {
            "query":     "$.updates[?(path=='srl_nokia-system:system/srl_nokia-system-info:information/last-booted')].val",
        },
        "interfaces": {
            "convert": "jsonpath",
            "query":   "$.updates[*].val['srl_nokia-interfaces:interface'][?(admin-state=='enable')]",
            "keep": True,
            "list": {
                "convert": "jsonpath",
                "query":   "name,admin-state,oper-state,description,mtu,last-change,ethernet",
                "index":   "name",
                "fields": {
                    "admin-state": {
                        "key":     "is_enabled",
                        "convert": "map",
                        "enable":  True,
                        "disable": False,
                    },
                    "oper-state":  {
                        "key":     "is_up",
                        "convert": "map",
                        "up":      True,
                        "down":    False,
                    },
                    "last-change":  {
                        "key":     "last_flapped",
                        "convert": "isotime2epoch",
                    },
                    "ethernet":  {
                        "convert": "jsonpath",
                        "query": "hw-mac-address,port-speed",
                        "keep": False,
                        "fields": {
                            "hw-mac-address": {
                                "key": "mac_address",
                            },
                            "port-speed": {
                                "key": "speed",
                                "convert": "map",
                                "1G":      1000.0,
                                "10G":    10000.0,
                                "25G":    25000.0,
                                "40G":    40000.0,
                                "50G":    50000.0,
                                "100G":  100000.0,
                                "200G":  200000.0,
                                "400G":  400000.0,
                                "800G":  800000.0,
                            },
                        },
                    },
                },                
            },
        },
    },
}


In [ ]:
node_data = {
    "name": "from_json",
    "type": "node-data",
    "nodes": {},
}
nodes = {}
results = []

with open('../node_data_srl.json', 'r', encoding='utf-8') as f:
    results.append(json.load(f))

In [ ]:
def json_convert_key(jcmap, key):
    k = key
    if "key" in jcmap.keys():
        k = jcmap["key"]
    return k

def json_convert_value(jcmap, value):
    v = value
    if "convert" in jcmap.keys():
        how = jcmap["convert"]
        if how == "map" and value in jcmap.keys():
            v = jcmap[value]
        elif how == "isotime2epoch":
            utc_time = datetime.strptime(value, "%Y-%m-%dT%H:%M:%S.%fZ")
            v = (utc_time - datetime(1970, 1, 1)).total_seconds()
        elif how == "jsonpath" and "query" in jcmap.keys() and isinstance(value, dict):
            matches = parse(jcmap["query"]).find(value)
            v = json_convert(jcmap, matches)

    return v

def json_convert(jcmap, matches):
    d = {}
    if "fields" in jcmap.keys():
        for m in matches:
            path = str(m.path)
            value = m.value
            if path in jcmap["fields"]:
                field = jcmap["fields"][path]
                k = json_convert_key(field, path)
                v = json_convert_value(field, value)
                if isinstance(v, dict) and "keep" in field.keys() and field["keep"] == False:
                    d |= v
                else:
                    d |= {k: v}
            else:
                d |= {path: value}
    elif "list" in jcmap.keys() and "query" in jcmap["list"]:
        exp = parse(jcmap["list"]["query"])
        for m in matches:
            submatches = exp.find(m)
            d |= json_convert(jcmap["list"], submatches)
        

    if "index" in jcmap.keys() and jcmap["index"] in d.keys():
        index = d.pop(jcmap["index"])
        return {index: d}
    else:
        return d

In [ ]:
def parse_results_gnmi_get_with_jsonpaths(kind, result):
    data = {"kind": kind}
    # flatten multiple updates into one list
    updates = {"updates": []}
    if "notification" in result:
        for n in result["notification"]:
            if "update" in n.keys():
                for u in n["update"]:
                  updates["updates"].append(u)

    for key, jp in kinds_jsonpaths[kind].items():
        matches = parse(jp["query"]).find(updates)
        data |= {key:{}}
        if "list" in jp.keys():
            data[key] |= json_convert(jp, matches)
                    
        elif len(matches) == 1:
            data |= {key: matches[0].value}
                            
    #print(json.dumps(updates, indent=4))
  
    return data

In [ ]:
%%time
n = {}
n |= parse_results_gnmi_get_with_jsonpaths("srl", results[0]["result"])
n |= {'vendor': 'Nokia'}

print(json.dumps(n, indent=4))